<a href="https://colab.research.google.com/github/Andron00e/Fine-Tuning-project/blob/main/Merge_Evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git
!pip install accelerate
!pip install bitsandbytes
!pip install sentencepiece
!pip install datasets
!pip uninstall wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
HUGGING_FACE_USER_NAME = "Andron00e"

model_name = "Open-Llama-3B-LoRA-OpenOrca_v2"

In [5]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = f"{HUGGING_FACE_USER_NAME}/{model_name}"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,  load_in_8bit=False, torch_dtype=torch.bfloat16,  device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
qa_model = PeftModel.from_pretrained(model, peft_model_id) # load LoRA adapters

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [6]:
merged_model = qa_model.merge_and_unload()

In [7]:
merged_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 3200, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (k_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (v_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (o_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3200, out_features=8640, bias=False)
          (down_proj): Linear(in_features=8640, out_features=3200, bias=False)
          (up_proj): Linear(in_features=3200, out_features=8640, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm(

In [8]:
merged_model_name = "YetAnother_Open-Llama-3B-LoRA-OpenOrca"

merged_model.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{merged_model_name}", use_auth_token=True)

pytorch_model.bin:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Andron00e/YetAnother_Open-Llama-3B-LoRA-OpenOrca/commit/0558f1a1c1514a04dcfc38ea18609b36843b9e9e', commit_message='Upload LlamaForCausalLM', commit_description='', oid='0558f1a1c1514a04dcfc38ea18609b36843b9e9e', pr_url=None, pr_revision=None, pr_num=None)

Evaluation

In [7]:
HUGGING_FACE_USER_NAME = "Andron00e"

In [9]:
merged_model_name = "YetAnother_Open-Llama-3B-LoRA-OpenOrca"
model_name = "Open-Llama-3B-LoRA-OpenOrca_v2"

In [10]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = f"{HUGGING_FACE_USER_NAME}/{model_name}"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,  load_in_8bit=False, torch_dtype=torch.bfloat16,  device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
qa_model = PeftModel.from_pretrained(model, peft_model_id) # load LoRA adapters

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [11]:
merged_model_name

'YetAnother_Open-Llama-3B-LoRA-OpenOrca'

In [12]:
tokenizer

LlamaTokenizerFast(name_or_path='openlm-research/open_llama_3b', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True)}, clean_up_tokenization_spaces=False)

In [13]:
tokenizer.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{merged_model_name}", use_auth_token=True)

tokenizer.model:   0%|          | 0.00/534k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Andron00e/YetAnother_Open-Llama-3B-LoRA-OpenOrca/commit/22e35cf0a0be1c0c1306be37955d897b12dfd0f3', commit_message='Upload tokenizer', commit_description='', oid='22e35cf0a0be1c0c1306be37955d897b12dfd0f3', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
from IPython.display import display, Markdown

def make_inference(system_prompt, question):
  batch = tokenizer(f"### CONTEXT\n{system_prompt}\n\n### QUESTION\n{question}\n\n### RESPONSE\n", return_tensors='pt')

  with torch.cuda.amp.autocast():
    output_tokens = qa_model.generate(**batch, max_new_tokens=200)

  display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))

In [16]:
!git clone https://github.com/EleutherAI/lm-evaluation-harness

Cloning into 'lm-evaluation-harness'...
remote: Enumerating objects: 14068, done.
remote: Counting objects: 100% (2480/2480), done.
remote: Compressing objects: 100% (459/459), done.
remote: Total 14068 (delta 2228), reused 2103 (delta 2019), pack-reused 11588
Receiving objects: 100% (14068/14068), 19.20 MiB | 10.25 MiB/s, done.
Resolving deltas: 100% (9343/9343), done.


In [17]:
%cd lm-evaluation-harness

/content/lm-evaluation-harness


In [18]:
!pip install -e ".[multilingual]"

Obtaining file:///content/lm-evaluation-harness
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 42.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6

In [4]:
%cd lm-evaluation-harness

/content/lm-evaluation-harness


In [5]:
!python main.py \
    --model hf-causal \
    --model_args pretrained=Andron00e/YetAnother_Open-Llama-3B-LoRA-OpenOrca \
    --tasks hellaswag \
    --device cuda:0

2023-07-19 07:00:07.753789: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[dynet] random seed: 1234
[dynet] allocating memory: 32MB
[dynet] memory allocation done.
Selected Tasks: ['hellaswag']
Using device 'cuda:0'
Extracting data files: 100% 3/3 [00:00<00:00, 1232.05it/s]
Dataset hellaswag downloaded and prepared to /root/.cache/huggingface/datasets/hellaswag/default/0.1.0/512a66dd8b1b1643ab4a48aa4f150d04c91680da6a4096498a5e5f799623d5ae. Subsequent calls will reuse this data.
100% 3/3 [00:00<00:00, 155.87it/s]
Task: hellaswag; number of docs: 10042
Task: hellaswag; document 0; context prompt (starting on next line):
Personal Care and Style: How to increase breast size with a bra. Check your bra size. Wearing a bra that is too big will not make your breasts look larger. That is why it is important to wear the right size bra for you.
(end of prompt on previous line)
Requests: [Req_loglikelihood('Personal Care and Style: How to increase 